In [12]:
import pandas as pd
import seaborn as sns
sns.set_theme()

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (13,9)

vacs = pd.read_csv("./Downloads/covid19-france-stock-vaccin-departement.csv", 
                   names = ['dept_code','dept_name','type','nb_dose','z1','date','z2','z3','iso3166-3','geo_point'], 
                   skiprows=1,
                   sep=';')
vacs = vacs.drop(labels=['z1','z2','z3'], axis=1)

# type conversions
vacs.date = pd.to_datetime(vacs.date, format="%Y-%m-%d")
vacs.type = pd.Categorical(vacs.type)

# date to run analysis on, few dates per month only
analysis_date = '2021-07-29'
vacs = vacs[vacs.date==analysis_date]

vacs.dtypes
vacs.tail()

ValueError: Can only compare identically-labeled Series objects

In [ ]:
dose_per_dept = vacs.groupby(['dept_code', 'dept_name'], as_index=False)['nb_dose'].sum()
dose_per_dept.nlargest(9, 'nb_dose')

In [ ]:
import geopandas
base = geopandas.read_file('./Downloads/departements-20180101-shp/departements-20180101.shp')

In [ ]:
# filter out three-letter dpt codes (outre-mer?) 
base = base[base.code_insee.str.len() <= 2]

In [ ]:
mg = pd.merge(base, dose_per_dept, left_on='code_insee', right_on='dept_code')
ax = mg.plot(column='nb_dose', cmap='YlGn', legend=True)
ax.set_title('Vaccin stock per dpt - ' + analysis_date)

In [ ]:
# df top9 departments with most vaccines
top9 = vacs[vacs.dept_code.isin(dose_per_dept.nlargest(9, 'nb_dose').dept_code)]

In [ ]:
# Initialize grid and map barplots
grid = sns.FacetGrid(top9, col='dept_name', hue='type', palette='pastel',
                     col_order = dose_per_dept.nlargest(9, 'nb_dose').dept_name,
                     col_wrap=3, height=4)
vac_list = ['Pfizer','AstraZeneca','Moderna','Janssen']
grid.map(sns.barplot,'type','nb_dose', order=vac_list)
grid.set_titles(col_template="{col_name}", row_template="{row_name}")
grid.add_legend(label_order=vac_list)